# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import time
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
y = df[[col for col in df.columns if 'category' in col]]

In [3]:
print(X.shape, y.shape)
df.head()

(26216,) (26216, 36)


,id,message,original,genre,category_related,category_request,category_offer,category_aid_related,category_medical_help,category_medical_products,...,category_aid_centers,category_other_infrastructure,category_weather_related,category_floods,category_storm,category_fire,category_earthquake,category_cold,category_other_weather,category_direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

**Before doing anything:** Notice that there are '2' in y, which does not make sense. I make the assumption to take the mode values, meaning that I replace all 2 by 0. 

In [6]:
# replace all 2 in y by 0 
for col in y.columns:
    if 2 in y[col].unique():
        print(col, 'contains unique values', y[col].unique(), 'where 2 is an unreasonable number.')
y = y.replace(2, 0)
print('After replacing, we have that the suspecious category now contains only unique values ', y['category_related'].unique())

category_related contains unique values [1 0 2] where 2 is an unreasonable number.
After replacing, we have that the suspecious category now contains only unique values  [1 0]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
print('The accuracy score is ', accuracy_score(y_test.values.flatten(), y_pred.flatten()))
print('\nThe classification report is \n', classification_report(y_test.values[:,], y_pred, target_names = y_test.columns))

The accuracy score is  0.941422540942

The classification report is 
                                  precision    recall  f1-score   support

               category_related       0.82      0.93      0.87      4962
               category_request       0.84      0.35      0.50      1141
                 category_offer       0.00      0.00      0.00        32
           category_aid_related       0.77      0.51      0.61      2744
          category_medical_help       0.62      0.04      0.08       567
      category_medical_products       0.81      0.09      0.16       341
     category_search_and_rescue       0.50      0.02      0.04       188
              category_security       1.00      0.02      0.03       118
              category_military       0.63      0.08      0.15       205
           category_child_alone       0.00      0.00      0.00         0
                 category_water       0.83      0.17      0.28       424
                  category_food       0.86      0.44 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [9]:
'''
for i in np.arange(y_pred.shape[1]):
    pred = np.transpose(y_pred)[i]
    true = y_test[df.columns[i+4]] # select the corrresponding column from y_test
    print(df.columns[i+4],'\n', classification_report(true, pred)) # print the report on each category
'''

"\nfor i in np.arange(y_pred.shape[1]):\n    pred = np.transpose(y_pred)[i]\n    true = y_test[df.columns[i+4]] # select the corrresponding column from y_test\n    print(df.columns[i+4],'\n', classification_report(true, pred)) # print the report on each category\n"

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
def build_model(pipeline = pipeline):
    '''
    build a model and use grid search to find better parameters
    Input:
        pipeline: the machine learning pipeline that has been built before
    Output:
        cv: from GridSearchCV
        (the best parameters are also printed)
    '''
    
    parameters = {
            'clf__estimator__n_estimators': [10, 20, 30],
            'clf__estimator__min_samples_split': [2, 4, 6],
        }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [11]:
begin = time.time()

model = build_model()
model.fit(X_train, y_train)
y_pred_tuned = model.predict(X_test)

finish = time.time()
print("Model training time = ", finish - begin, " seconds.")

Model training time =  2464.4249551296234  seconds.


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
print("\nBest Parameters:", model.best_params_)
print('After tuning, the accuracy score is ', accuracy_score(y_test.values.flatten(), y_pred_tuned.flatten()))
print('\nThe classification report is \n', classification_report(y_test.values[:,], y_pred_tuned, target_names = y_test.columns))


Best Parameters: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 30}
After tuning, the accuracy score is  0.944457159326

The classification report is 
                                  precision    recall  f1-score   support

               category_related       0.80      0.97      0.88      4962
               category_request       0.86      0.40      0.55      1141
                 category_offer       0.00      0.00      0.00        32
           category_aid_related       0.77      0.61      0.68      2744
          category_medical_help       0.65      0.06      0.11       567
      category_medical_products       0.81      0.08      0.14       341
     category_search_and_rescue       0.71      0.06      0.12       188
              category_security       0.67      0.02      0.03       118
              category_military       0.73      0.05      0.10       205
           category_child_alone       0.00      0.00      0.00         0
                 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# let's try using bagging with knn
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)
pipeline_bag = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('bag', MultiOutputClassifier(estimator=bagging))
    ])

begin = time.time()

pipeline_bag.fit(X_train, y_train)
y_pred_bag = pipeline_bag.predict(X_test)

finish = time.time()
print("Model training time = ", finish - begin, " seconds.")

print('The accuracy score is ', accuracy_score(y_test.values.flatten(), y_pred_bag.flatten()))
print('\nThe classification report is \n', classification_report(y_test.values[:,], y_pred_bag, target_names = y_test.columns))

Model training time =  1176.4351541996002  seconds.
The accuracy score is  0.931153154986

The classification report is 
                                  precision    recall  f1-score   support

               category_related       0.81      0.91      0.86      4962
               category_request       0.90      0.28      0.43      1141
                 category_offer       0.00      0.00      0.00        32
           category_aid_related       0.87      0.14      0.24      2744
          category_medical_help       0.50      0.00      0.00       567
      category_medical_products       1.00      0.01      0.02       341
     category_search_and_rescue       0.00      0.00      0.00       188
              category_security       0.00      0.00      0.00       118
              category_military       0.00      0.00      0.00       205
           category_child_alone       0.00      0.00      0.00         0
                 category_water       0.92      0.03      0.05       424
 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
import pickle
def save_model(model, model_filepath):
    """ Saving model's best_estimator_ using pickle
    """
    pickle.dump(model, open(model_filepath, 'wb'))
    
save_model(model, 'model.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.